# Lab 06 Softmax Zoo_classifier-eager

* Softmax를 사용하여 Zoo 데이터를 활용하여 분류를 진행합니다.

### 기본 Library 선언 및 Tensorflow 버전 확인

In [53]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

ValueError: tf.enable_eager_execution must be called at program startup.

In [54]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(list(y_data), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

(101, 16) (101, 1)


In [55]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, Y_one_hot)).batch(16).repeat(100)

In [56]:
#Weight and bias setting
W = tfe.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

def cost_fn(X, Y):
    logits = hypothesis(X)
    cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                           labels=tf.stop_gradient(Y))
    cost = tf.reduce_mean(cost_i)
    
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [60]:
def fit(X, Y, epochs=5000, verbose=500):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.05)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
#             print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy()
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 1.4284837245941162, Acc: 0.7326732873916626
Steps: 500 Loss: 1.4279112815856934, Acc: 0.7326732873916626
Steps: 1000 Loss: 1.4274009466171265, Acc: 0.7326732873916626
Steps: 1500 Loss: 1.4269442558288574, Acc: 0.7326732873916626
Steps: 2000 Loss: 1.426532506942749, Acc: 0.7326732873916626
Steps: 2500 Loss: 1.4261595010757446, Acc: 0.7326732873916626
Steps: 3000 Loss: 1.4258196353912354, Acc: 0.7326732873916626
Steps: 3500 Loss: 1.4255083799362183, Acc: 0.7326732873916626
Steps: 4000 Loss: 1.425222396850586, Acc: 0.7326732873916626
Steps: 4500 Loss: 1.4249582290649414, Acc: 0.7326732873916626
Steps: 5000 Loss: 1.4247134923934937, Acc: 0.7326732873916626


# For Class

In [31]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tfe.Variable(tf.random_normal([16, nb_classes]), name='weight')
        self.b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        # tf.nn.softmax computes softmax activations
        # softmax = exp(logits) / reduce_sum(exp(logits), dim)
        logits = tf.matmul(X, self.W) + self.b
        return tf.nn.softmax(logits)
    
    def loss_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                           labels=Y)
        cost = tf.reduce_mean(cost_i)
        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            loss = self.loss_fn(X, Y)
            grads = tape.gradient(loss, self.variables)
            
            return grads
    
    def prediction(self, X, Y):
        pred = tf.argmax(self.softmax_regression(X), 1)
        correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        return accuracy
    
    def fit(self, dataset, epochs=1001, verbose=50):
        optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)
        
        for i in range(epochs):
            step = 0
            
            for features, labels in tfe.Iterator(dataset):
                
                step += 1
                
                grads = self.grad_fn(features, labels)
                optimizer.apply_gradients(zip(grads, self.variables))
                
                if step % verbose == 0:
                    acc = self.prediction(features, labels).numpy()
                    loss = self.loss_fn(features, labels).numpy()
                    print('Steps: {} Loss: {}, Acc: {}'.format(step, loss, acc))

In [32]:
model = softmax_classifer(nb_classes)
model.fit(dataset)

Steps: 50 Loss: 2.003817558288574, Acc: 0.1875
Steps: 100 Loss: 2.1641604900360107, Acc: 0.0
Steps: 150 Loss: 2.0985264778137207, Acc: 0.0625
Steps: 200 Loss: 2.098926305770874, Acc: 0.0625
Steps: 250 Loss: 2.0350992679595947, Acc: 0.125
Steps: 300 Loss: 2.0418856143951416, Acc: 0.125
Steps: 350 Loss: 1.9834251403808594, Acc: 0.20000000298023224
Steps: 400 Loss: 1.9944957494735718, Acc: 0.1875
Steps: 450 Loss: 2.1639621257781982, Acc: 0.0
Steps: 500 Loss: 2.0972094535827637, Acc: 0.0625
Steps: 550 Loss: 2.0975914001464844, Acc: 0.0625
Steps: 600 Loss: 2.022022247314453, Acc: 0.1875
Steps: 650 Loss: 2.0380849838256836, Acc: 0.125
Steps: 700 Loss: 1.9773708581924438, Acc: 0.20000000298023224
Steps: 50 Loss: 1.9882460832595825, Acc: 0.1875
Steps: 100 Loss: 2.1633596420288086, Acc: 0.0
Steps: 150 Loss: 2.096513271331787, Acc: 0.0625
Steps: 200 Loss: 2.0957467555999756, Acc: 0.0625
Steps: 250 Loss: 2.009125232696533, Acc: 0.1875
Steps: 300 Loss: 2.035257339477539, Acc: 0.125
Steps: 350 Loss

Steps: 50 Loss: 1.4675233364105225, Acc: 0.6875
Steps: 100 Loss: 1.6991726160049438, Acc: 0.4375
Steps: 150 Loss: 1.7020220756530762, Acc: 0.4375
Steps: 200 Loss: 1.6499699354171753, Acc: 0.5
Steps: 250 Loss: 1.468111515045166, Acc: 0.6875
Steps: 300 Loss: 1.652348279953003, Acc: 0.5
Steps: 350 Loss: 1.5467439889907837, Acc: 0.6000000238418579
Steps: 400 Loss: 1.4723237752914429, Acc: 0.6875
Steps: 450 Loss: 1.635450839996338, Acc: 0.5625
Steps: 500 Loss: 1.6394238471984863, Acc: 0.5
Steps: 550 Loss: 1.652381181716919, Acc: 0.5
Steps: 600 Loss: 1.4679474830627441, Acc: 0.6875
Steps: 650 Loss: 1.628553032875061, Acc: 0.5625
Steps: 700 Loss: 1.391684651374817, Acc: 0.800000011920929
Steps: 50 Loss: 1.5001506805419922, Acc: 0.625
Steps: 100 Loss: 1.588355541229248, Acc: 0.5625
Steps: 150 Loss: 1.5706688165664673, Acc: 0.625
Steps: 200 Loss: 1.6253489255905151, Acc: 0.5
Steps: 250 Loss: 1.46742844581604, Acc: 0.6875
Steps: 300 Loss: 1.6105856895446777, Acc: 0.5625
Steps: 350 Loss: 1.381812

Steps: 600 Loss: 1.4629029035568237, Acc: 0.6875
Steps: 650 Loss: 1.5909807682037354, Acc: 0.5625
Steps: 700 Loss: 1.3591105937957764, Acc: 0.800000011920929
Steps: 50 Loss: 1.465217113494873, Acc: 0.6875
Steps: 100 Loss: 1.5830748081207275, Acc: 0.5625
Steps: 150 Loss: 1.525376319885254, Acc: 0.625
Steps: 200 Loss: 1.5874543190002441, Acc: 0.5625
Steps: 250 Loss: 1.4628537893295288, Acc: 0.6875
Steps: 300 Loss: 1.5907427072525024, Acc: 0.5625
Steps: 350 Loss: 1.3588892221450806, Acc: 0.800000011920929
Steps: 400 Loss: 1.4651236534118652, Acc: 0.6875
Steps: 450 Loss: 1.5829501152038574, Acc: 0.5625
Steps: 500 Loss: 1.5251762866973877, Acc: 0.625
Steps: 550 Loss: 1.5873186588287354, Acc: 0.5625
Steps: 600 Loss: 1.4628077745437622, Acc: 0.6875
Steps: 650 Loss: 1.5905179977416992, Acc: 0.5625
Steps: 700 Loss: 1.3586770296096802, Acc: 0.800000011920929
Steps: 50 Loss: 1.4650342464447021, Acc: 0.6875
Steps: 100 Loss: 1.582829475402832, Acc: 0.5625
Steps: 150 Loss: 1.5249848365783691, Acc: 0

KeyboardInterrupt: 